In [3]:
%pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   ---------- ----------------------------- 41.0/163.3 kB ? eta -:--:--
   ------------------------------ --------- 122.9/163.3 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 163.3/163.3 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/107.1 kB ? eta -:--:--
   --------------- ------------------------ 41.0/107.1 kB ? eta -:--:--
   ---------------------------------------- 107.1/107.1 kB 1.2 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Note: you may need to restart the kernel to use updated pac

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### This script deletes empty semantic models that are migrated/decoupled from a Microsoft Fabric workspace
 
### Overview:
#### 1. Set up the workspace ID and API endpoint for migrated/decoupled semantic models
#### 2. Configure authentication headers with bearer token. Get the bearer token manually from developer tools or include logic to fetch access token.
#### 3. Fetch all decoupled default semantic models from the specified workspace
#### 4. Iterate through models and delete only the empty ones
#### 5. Delete and provide deletion status for each model

### Note: Replace <ACCESS_TOKEN> in the headers with your actual access token
### Warning: This will permanently delete empty decoupled semantic models - use with caution

In [3]:
import requests

In [ ]:
workspace_id = "your_workspace_id_here"
migratedsemanticmodels_url = f"https://api.fabric.microsoft.com/v1.0/myorg/groups/{workspace_id}/datawarehouses/migratedSemanticModels"

In [ ]:
payload = ""
headers = {
    "User-Agent": "delete-default-semantic-models-script/1.0",
    "Authorization": "Bearer <ACCESS_TOKEN>",
}

In [ ]:
response = requests.request("GET", migratedsemanticmodels_url, headers=headers)
response.raise_for_status()

models = response.json().get("value", [])
print(f"Found {len(models)} semantic models.")

In [ ]:
# Iterate and delete each model
for model in models:
    model_id = model.get("modelObjectId")
    is_empty = model.get("isEmpty")
    delete_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/semanticModels/{model_id}"
    if is_empty:
        print(f"Would delete (empty): ({model_id}) - URL: {delete_url}")
        delete_response = requests.delete(delete_url, headers=headers)
        if delete_response.status_code == 200:
            print(f"✅ Deleted: ({model_id})")
        elif delete_response.status_code == 404:
            print(f"⚠️ Not found or already deleted: ({model_id})")
        else:
            print(f"❌ Failed to delete ({model_id}): {delete_response.status_code}")
    else:
        print(f"Skipping non-empty model: ({model_id})")